# Run GNN Model

This notebook illustrates how to build a GNN model, load its training checkpoint, and run it.

In [1]:
import torch
import pytorch_lightning as pl
import torch_geometric

import models
import model_utils

Load Data

In [2]:
from data_utils import read_data, create_data

node_dataset, edge_list, labels = read_data(node_filepath="../data/final_data/node_node2vec_data.csv",
                                            label_filepath="../data/final_data/training_labels_trials.csv",
                                            edgelist_path="../data/final_data/ls-fgin_edge_list.edg",
                                            feats_type='nodeonly')
data = create_data(node_dataset, edge_list, labels, label_col='label_99') # using labels associated with 99th trial

In [3]:
num_classes = 2
num_features = len(node_dataset.columns)

Load Model

In [4]:
# select model

model_name = 'SGConv' # options: 'SGConv', 'GraphSAGE', 'TAG', 'ClusterGCN'

model_creator_dict = {'SGConv': models.create_SGConv_GNN, 'GraphSAGE': models.create_GraphSAGE_GNN, 'TAG': models.create_TAG_GNN}

path_to_checkpoint_dict = {'SGConv': 'model_checkpoints/SGConv_trial99.ckpt'}

In [5]:
create_model = model_creator_dict[model_name]

In [6]:
model = create_model(model_name, num_features=num_features, num_classes=num_classes)

In [7]:
path_to_checkpoint = path_to_checkpoint_dict[model_name]
model.load_from_checkpoint(path_to_checkpoint)

LitGNN(
  (model): GNNModel(
    (convs): ModuleList(
      (0): SGConv(107, 128, K=1)
      (1): SGConv(128, 256, K=1)
      (2): SGConv(256, 256, K=1)
      (3): SGConv(256, 128, K=1)
    )
    (dense1): Linear(in_features=128, out_features=128, bias=True)
    (dense_out): Linear(in_features=128, out_features=2, bias=True)
  )
  (loss_module): CrossEntropyLoss()
)

Run model on data

In [ ]:
logits, _, _ = model.forward(data.to(device='cuda'))

In [ ]:
train_report, test_report = model_utils.evaluate_model(model, data)